In [21]:
from statsbombpy import sb
import pandas as pd

In [68]:
# dataframe for world cup 2022
world_cup = sb.matches(competition_id=43, season_id=106)

world_cup_events = sb.competition_events(
    country="International",
    division="FIFA World Cup",
    season="2022",
)

# get all goal events - these are where shot_outcome is 'Goal'
world_cup_goals = world_cup_events[world_cup_events['shot_outcome'] == 'Goal']

/Users/ashlanahmed/school/4/Code Data Art/final/cda-final/lib/python3.11/site-packages/statsbombpy/api_client.py:21: NoAuthWarning: credentials were not supplied. open data access only
  warnings.warn(
/Users/ashlanahmed/school/4/Code Data Art/final/cda-final/lib/python3.11/site-packages/statsbombpy/api_client.py:21: NoAuthWarning: credentials were not supplied. open data access only
  warnings.warn(
/Users/ashlanahmed/school/4/Code Data Art/final/cda-final/lib/python3.11/site-packages/statsbombpy/api_client.py:21: NoAuthWarning: credentials were not supplied. open data access only
  warnings.warn(
/Users/ashlanahmed/school/4/Code Data Art/final/cda-final/lib/python3.11/site-packages/statsbombpy/api_client.py:21: NoAuthWarning: credentials were not supplied. open data access only
  warnings.warn(
/Users/ashlanahmed/school/4/Code Data Art/final/cda-final/lib/python3.11/site-packages/statsbombpy/api_client.py:21: NoAuthWarning: credentials were not supplied. open data access only
  warni

In [292]:
# returns a list of lists, with each sublist being the path to a goal
def get_goal_path(match_id):
    match_events_unsorted = world_cup_events[world_cup_events['match_id'] == match_id]
    # sort by period first, then within each period by minute, then within each minute by second - then reverse the whole thing
    match_events = match_events_unsorted.sort_values(by=['index', 'period', 'minute', 'second'], ascending=[False, False, False, False]) 

    # remove duplicates (rows with the same value in index column)
    match_events = match_events.drop_duplicates(subset=['index'])
    
    # return match_events[['index', 'period', 'minute', 'second', 'possession_team', 'player', 'position', 'type', 'shot_outcome', 'location']]

    goal_paths = []
    turnover_goal = False

    # iterate through match_events and when we find a goal (has type as 'Shot' and shot_outcome as 'Goal'), add each subsequent event to curr_goal_path until we find type = Half Start or posession_team is different from goal scoring team
    curr_goal_path = []


    for index, row in match_events.iterrows():
        if row['type'] == 'Shot' and row['shot_outcome'] == 'Goal':
            curr_goal_path.append(row)
            if row['possession_team'] != row['team']:
                # means the goal was scored on a turnover (scoring team didn't have possession), so we should just iterate back until the team changed
                turnover_goal = True
        
        if curr_goal_path != []: # if we're in the middle of a goal path
            if turnover_goal:
                if row['team'] != curr_goal_path[-1]['team']:
                    goal_paths.append(curr_goal_path)
                    curr_goal_path = []
                    turnover_goal = False
                else:
                    curr_goal_path.append(row)
            else:
                if row['type'] == 'Half Start' or row['possession_team'] != curr_goal_path[-1]['possession_team']:
                    goal_paths.append(curr_goal_path)
                    curr_goal_path = []
                else:
                    curr_goal_path.append(row)

    curr_goal_num = len(goal_paths)
    # convert each goal path to a dataframe, and add a column for each with the goal number
    for i in range(len(goal_paths)):
        goal_paths[i] = pd.DataFrame(goal_paths[i])
        goal_paths[i]['goal_num'] = curr_goal_num
        curr_goal_num -= 1

        # if the first row has team equal possession_team, then remove every row in this dataframe that does not have team equal possession_team
        if goal_paths[i].iloc[0]['team'] == goal_paths[i].iloc[0]['possession_team']:
            goal_paths[i] = goal_paths[i][goal_paths[i]['team'] == goal_paths[i]['possession_team']]

    return goal_paths # reverse the list of goal paths so that the first goal is first in the list


In [294]:
first_row_id = world_cup.iloc[52]['match_id']
# print the match date, home team, home team score, away team, away team score
print(world_cup[world_cup['match_id'] == first_row_id][['match_date', 'competition_stage', 'home_team', 'home_score', 'away_team', 'away_score']].iloc[0])

test_goal_paths = get_goal_path(first_row_id)

# test_goal_paths.head(20)



print(len(test_goal_paths))

# flatten the dataframes in test_goal_paths
for i in range(len(test_goal_paths)):
    test_goal_paths[i] = test_goal_paths[i].reset_index(drop=True)


test_goal_paths[0][['index', 'period', 'timestamp', 'minute', 'second', 'possession_team', 'team', 'player', 'position', 'type', 'shot_outcome', 'location', 'play_pattern', 'goal_num']]

# write this as a csv
# test_goal_paths[0].to_csv('test.csv')

match_date            2022-12-01
competition_stage    Group Stage
home_team                 Canada
home_score                     1
away_team                Morocco
away_score                     2
Name: 52, dtype: object
2


,index,period,timestamp,minute,second,possession_team,team,player,position,type,shot_outcome,location,play_pattern,goal_num
0,845,1,00:22:36.384,22,36,Morocco,Morocco,Youssef En-Nesyri,Center Forward,Shot,Goal,"[107.6, 53.2]",From Throw In,2
1,845,1,00:22:36.384,22,36,Morocco,Morocco,Youssef En-Nesyri,Center Forward,Shot,Goal,"[107.6, 53.2]",From Throw In,2
2,843,1,00:22:35.188,22,35,Morocco,Morocco,Youssef En-Nesyri,Center Forward,Carry,NaN,"[97.9, 56.4]",From Throw In,2
3,842,1,00:22:35.188,22,35,Morocco,Morocco,Youssef En-Nesyri,Center Forward,Ball Receipt*,NaN,"[97.9, 56.4]",From Throw In,2
4,840,1,00:22:31.828,22,31,Morocco,Morocco,Achraf Hakimi Mouh,Right Back,Pass,NaN,"[39.0, 79.2]",From Throw In,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87,750,1,00:20:32.051,20,32,Morocco,Morocco,Noussair Mazraoui,Left Back,Ball Receipt*,NaN,"[34.7, 30.0]",From Throw In,2
88,749,1,00:20:29.248,20,29,Morocco,Morocco,Youssef En-Nesyri,Center Forward,Pass,NaN,"[34.8, 70.4]",From Throw In,2
89,748,1,00:20:29.208,20,29,Morocco,Morocco,Youssef En-Nesyri,Center Forward,Carry,NaN,"[34.7, 70.4]",From Throw In,2
90,747,1,00:20:29.208,20,29,Morocco,Morocco,Youssef En-Nesyri,Center Forward,Ball Receipt*,NaN,"[34.7, 70.4]",From Throw In,2


In [373]:
# get all the goal_paths for all the matches in the world cup - give each entry in a goal path a unique id called goal_id (start from 0 and increment
# goal here is to have one big CSV with all the goal paths in it, with all rows associated with the same goal path having the same goal_id
all_goal_paths = []
goal_id = 1
for match_id in world_cup['match_id']:
    match_goal_paths = get_goal_path(match_id)
    for goal_path in match_goal_paths:
        goal_id += 1
    all_goal_paths.extend(match_goal_paths)

# flatten the dataframes in all_goal_paths
for i in range(len(all_goal_paths)):
    all_goal_paths[i] = all_goal_paths[i].reset_index(drop=True)

# modify all_goal_paths so that every goal has a unique id called goal_id, starting from 1 and incrementing.

curr_goal_id = 1

curr_match_id = None
curr_goal_num = 0
for i in range(len(all_goal_paths)):
    if curr_match_id is None:
        # first row, so by default curr_match_id is None
        curr_match_id = all_goal_paths[i]['match_id'].iloc[0]
        curr_goal_num = all_goal_paths[i]['goal_num'].iloc[0]
        all_goal_paths[i]['goal_id'] = curr_goal_id

    # when we have a new unique goal, increment curr_goal_id and set curr_match_id and curr_goal_num to the new values
    if (all_goal_paths[i]['match_id'].iloc[0] != curr_match_id) or (all_goal_paths[i]['goal_num'].iloc[0] != curr_goal_num):
        curr_match_id = all_goal_paths[i]['match_id'].iloc[0]
        curr_goal_num = all_goal_paths[i]['goal_num'].iloc[0]
        curr_goal_id += 1
        all_goal_paths[i]['goal_id'] = curr_goal_id

    # otherwise we are still on the same goal, so just set goal_id to curr_goal_id
    else:
        all_goal_paths[i]['goal_id'] = curr_goal_id

# flatten the dataframes in all_goal_paths
for i in range(len(all_goal_paths)):
    all_goal_paths[i] = all_goal_paths[i].reset_index(drop=True)


# convert all_goal_paths to a dataframe
all_goal_paths_df = pd.concat(all_goal_paths)

# for all rows with the same match_id, get 'match_date', 'competition_stage', 'home_team', 'home_score', 'away_team', 'away_score' for them by looking at world_cup dataframe
# then add these columns to all_goal_paths_df for each row with the same match_id
# Assuming 'match_id' is a column in both all_goal_paths_df and world_cup dataframes
for match_id in all_goal_paths_df['match_id'].unique():
    match_info = world_cup[world_cup['match_id'] == match_id]

    all_goal_paths_df.loc[all_goal_paths_df['match_id'] == match_id, 'match_date'] = match_info['match_date'].iloc[0]
    all_goal_paths_df.loc[all_goal_paths_df['match_id'] == match_id, 'competition_stage'] = match_info['competition_stage'].iloc[0]
    all_goal_paths_df.loc[all_goal_paths_df['match_id'] == match_id, 'home_team'] = match_info['home_team'].iloc[0]
    all_goal_paths_df.loc[all_goal_paths_df['match_id'] == match_id, 'home_score'] = match_info['home_score'].iloc[0]
    all_goal_paths_df.loc[all_goal_paths_df['match_id'] == match_id, 'away_team'] = match_info['away_team'].iloc[0]
    all_goal_paths_df.loc[all_goal_paths_df['match_id'] == match_id, 'away_score'] = match_info['away_score'].iloc[0]


/var/folders/zq/zt0x8p7j2bxbmygnz7b0q8gc0000gn/T/ipykernel_64739/1312504136.py:53: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  all_goal_paths_df.loc[all_goal_paths_df['match_id'] == match_id, 'match_date'] = match_info['match_date'].iloc[0]
/var/folders/zq/zt0x8p7j2bxbmygnz7b0q8gc0000gn/T/ipykernel_64739/1312504136.py:54: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  all_goal_paths_df.loc[all_goal_paths_df['match_id'] == match_id, 'competition_stage'] = match_info['competition_stage'].iloc[0]
/var/folders/zq/zt0x8p7j2bxbmygnz7

In [374]:
# all_goal_paths_df[['match_id', 'period', 'minute', 'second', 'possession_team', 'team', 'player', 'position', 'type', 'shot_outcome', 'location', 'play_pattern', 'shot_end_location', 'shot_statsbomb_xg', 'shot_type', 'pass_height']]
all_goal_paths_df.head(20)[['goal_id', 'match_date', 'competition_stage', 'home_team', 'home_score', 'away_team', 'away_score', 'goal_num', 'period', 'timestamp', 'team', 'player', 'position', 'type', 'shot_outcome', 'location', 'play_pattern', 'shot_end_location', 'shot_statsbomb_xg', 'shot_type', 'pass_height']]
# len(all_goal_paths_df)

,goal_id,match_date,competition_stage,home_team,home_score,away_team,away_score,goal_num,period,timestamp,...,player,position,type,shot_outcome,location,play_pattern,shot_end_location,shot_statsbomb_xg,shot_type,pass_height
0,1,2022-12-02,Group Stage,Serbia,2.0,Switzerland,3.0,5,2,00:02:38.859,...,Remo Freuler,Right Defensive Midfield,Shot,Goal,"[107.2, 41.2]",From Free Kick,"[120.0, 42.7, 0.6]",0.153815,Open Play,NaN
1,1,2022-12-02,Group Stage,Serbia,2.0,Switzerland,3.0,5,2,00:02:38.859,...,Remo Freuler,Right Defensive Midfield,Shot,Goal,"[107.2, 41.2]",From Free Kick,"[120.0, 42.7, 0.6]",0.153815,Open Play,NaN
2,1,2022-12-02,Group Stage,Serbia,2.0,Switzerland,3.0,5,2,00:02:38.822,...,Remo Freuler,Right Defensive Midfield,Ball Receipt*,NaN,"[107.2, 41.2]",From Free Kick,NaN,NaN,NaN,NaN
3,1,2022-12-02,Group Stage,Serbia,2.0,Switzerland,3.0,5,2,00:02:37.854,...,Ruben Vargas,Left Wing,Pass,NaN,"[110.7, 49.5]",From Free Kick,NaN,NaN,NaN,Low Pass
4,1,2022-12-02,Group Stage,Serbia,2.0,Switzerland,3.0,5,2,00:02:37.854,...,Ruben Vargas,Left Wing,Ball Receipt*,NaN,"[110.7, 49.5]",From Free Kick,NaN,NaN,NaN,NaN
5,1,2022-12-02,Group Stage,Serbia,2.0,Switzerland,3.0,5,2,00:02:36.392,...,Xherdan Shaqiri,Right Wing,Pass,NaN,"[94.4, 59.6]",From Free Kick,NaN,NaN,NaN,High Pass
6,1,2022-12-02,Group Stage,Serbia,2.0,Switzerland,3.0,5,2,00:02:34.825,...,Xherdan Shaqiri,Right Wing,Carry,NaN,"[98.7, 60.9]",From Free Kick,NaN,NaN,NaN,NaN
7,1,2022-12-02,Group Stage,Serbia,2.0,Switzerland,3.0,5,2,00:02:34.825,...,Xherdan Shaqiri,Right Wing,Ball Receipt*,NaN,"[98.7, 60.9]",From Free Kick,NaN,NaN,NaN,NaN
8,1,2022-12-02,Group Stage,Serbia,2.0,Switzerland,3.0,5,2,00:02:33.092,...,Breel-Donald Embolo,Center Forward,Pass,NaN,"[105.5, 57.4]",From Free Kick,NaN,NaN,NaN,Ground Pass
9,1,2022-12-02,Group Stage,Serbia,2.0,Switzerland,3.0,5,2,00:02:30.483,...,Breel-Donald Embolo,Center Forward,Carry,NaN,"[106.6, 54.5]",From Free Kick,NaN,NaN,NaN,NaN


In [375]:
# remove all rows where the row following 
# all_goal_paths_df = all_goal_paths_df.drop_duplicates()
# write this as a csv
all_goal_paths_df[['goal_id', 'match_date', 'competition_stage', 'home_team', 'home_score', 'away_team', 'away_score', 'period', 'minute', 'second', 'team', 'player', 'position', 'type', 'shot_outcome', 'location', 'play_pattern', 'shot_end_location', 'shot_statsbomb_xg', 'shot_type', 'pass_height', 'goal_num']].to_csv('all_goal_paths.csv')

In [376]:
# read from the same csv, delete the first column, remove duplicates, and write to a new csv
all_goal_paths_df = pd.read_csv('all_goal_paths.csv')
del all_goal_paths_df['Unnamed: 0']
all_goal_paths_df = all_goal_paths_df.drop_duplicates()

# reverse the dataframe
all_goal_paths_df = all_goal_paths_df.iloc[::-1]

# remove all rows with type Substitution
all_goal_paths_df = all_goal_paths_df[all_goal_paths_df['type'] != 'Substitution']

# remove all rows with type Player On
all_goal_paths_df = all_goal_paths_df[all_goal_paths_df['type'] != 'Player On']

# remove all rows with type Tactical Shift
all_goal_paths_df = all_goal_paths_df[all_goal_paths_df['type'] != 'Tactical Shift']

# remove all rows with type Referee Ball-Drop
all_goal_paths_df = all_goal_paths_df[all_goal_paths_df['type'] != 'Referee Ball-Drop']

# remove all rows with type Bad Behaviour
all_goal_paths_df = all_goal_paths_df[all_goal_paths_df['type'] != 'Bad Behaviour']

# make the goal_id column be 196 - goal_id
all_goal_paths_df['goal_id'] = 196 - all_goal_paths_df['goal_id']

# fix the first column to be row number with no gaps from the deletion of duplicates
all_goal_paths_df = all_goal_paths_df.reset_index(drop=True)

all_goal_paths_df.to_csv('all_goal_paths.csv')

In [377]:
# give me all the unique values in the type column for the all_goal_paths_df dataframe
all_goal_paths_df['team'].unique()

array(['Spain', 'Japan', 'England', 'Portugal', 'South Korea', 'Belgium',
       'Iran', 'Netherlands', 'Ecuador', 'Tunisia', 'Morocco',
       'United States', 'Australia', 'France', 'Cameroon', 'Canada',
       'Croatia', 'Switzerland', 'Germany', 'Costa Rica', 'Uruguay',
       'Poland', 'Ghana', 'Argentina', 'Saudi Arabia', 'Senegal', 'Qatar',
       'Wales', 'Serbia', 'Brazil', 'Denmark', 'Mexico'], dtype=object)

In [389]:
# still preserving groupings of the same goal_id (no reordering within a chunk with same goal_id), sort by match_date
new_all_goal_paths_df = all_goal_paths_df.sort_values(by=['match_date', 'goal_id'], ascending=[True, True])


In [390]:
# fix the first column to be row number with no gaps from the deletion of duplicates
new_all_goal_paths_df = new_all_goal_paths_df.reset_index(drop=True)

# modify the values in goal_id column so that for each row, when we see a new goal_id, we increment our new goal_id_counter by 1 and make the value in the goal_id column be the new goal_id_counter
new_goal_id_counter = 0
seen = set()
for i in range(len(new_all_goal_paths_df)):
    if new_all_goal_paths_df.iloc[i]['goal_id'] not in seen:
        seen.add(new_all_goal_paths_df.iloc[i]['goal_id'])
        new_goal_id_counter += 1
        new_all_goal_paths_df.loc[i, 'goal_id'] = new_goal_id_counter
    else:
        new_all_goal_paths_df.loc[i, 'goal_id'] = new_goal_id_counter

# write to csv
new_all_goal_paths_df.to_csv('new_all_goal_paths.csv')

In [ ]:
# errors: argentina vs. australia is messed up (we only detect 2 goals when there were 3) - probably one other mess up we're missing since get 169 instead of 172 goals

# notes: we're filtering deeper than possessions from stats bomb (which would count throw in for same team as a new possession - full continuous flow to see what build up to a goal really looks like)

In [163]:
# get all world_cup_goals, filtering out penalties (has period of 5)

world_cup_goals_filtered = world_cup_goals[world_cup_goals['period'] != 5]


# write all of this to a csv
world_cup_goals_filtered.to_csv('world_cup_goals.csv')

In [164]:
# now make this csv a dataframe
world_cup_goals_df = pd.read_csv('world_cup_goals.csv')
world_cup_goals_df

,Unnamed: 0,50_50,bad_behaviour_card,ball_receipt_outcome,ball_recovery_offensive,ball_recovery_recovery_failure,block_deflection,block_offensive,block_save_block,carry_end_location,...,shot_technique,shot_type,substitution_outcome,substitution_replacement,tactics,team,team_id,timestamp,type,under_pressure
0,194920,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Normal,Open Play,NaN,NaN,NaN,Switzerland,773,00:19:11.927,Shot,NaN
1,194923,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Normal,Open Play,NaN,NaN,NaN,Serbia,786,00:25:56.269,Shot,NaN
2,194925,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Normal,Open Play,NaN,NaN,NaN,Serbia,786,00:34:28.223,Shot,NaN
3,194927,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Normal,Open Play,NaN,NaN,NaN,Switzerland,773,00:43:41.456,Shot,NaN
4,194929,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Half Volley,Open Play,NaN,NaN,NaN,Switzerland,773,00:02:38.859,Shot,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
164,196352,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Normal,Open Play,NaN,NaN,NaN,England,768,00:05:53.871,Shot,NaN
165,196354,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Normal,Open Play,NaN,NaN,NaN,England,768,00:22:40.407,Shot,NaN
166,196368,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Normal,Open Play,NaN,NaN,NaN,Spain,772,00:11:00.474,Shot,True
167,196372,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Normal,Open Play,NaN,NaN,NaN,Japan,778,00:02:48.337,Shot,NaN


In [169]:
# for each goal (row) in world_cup_goals_df, just grab: match_id, period, minute, second, possession_team, team, player, position, type, shot_outcome, location, play_pattern, shot_end_location, shot_statsbomb_xg, shot_type, pass_height

# get all the columns we want
world_cup_goals_df = world_cup_goals_df[['match_id', 'period', 'minute', 'second', 'possession_team', 'team', 'player', 'position', 'type', 'shot_outcome', 'location', 'play_pattern', 'shot_end_location', 'shot_statsbomb_xg', 'shot_type', 'pass_height']]
world_cup_goals_df

,match_id,period,minute,second,possession_team,team,player,position,type,shot_outcome,location,play_pattern,shot_end_location,shot_statsbomb_xg,shot_type,pass_height
0,3857256,1,19,11,Switzerland,Switzerland,Xherdan Shaqiri,Right Wing,Shot,Goal,"[107.5, 50.6]",From Free Kick,"[120.0, 43.1, 0.2]",0.069699,Open Play,NaN
1,3857256,1,25,56,Serbia,Serbia,Aleksandar Mitrović,Left Center Forward,Shot,Goal,"[108.3, 38.6]",Regular Play,"[120.0, 43.3, 0.9]",0.063718,Open Play,NaN
2,3857256,1,34,28,Serbia,Serbia,Dušan Vlahović,Right Center Forward,Shot,Goal,"[106.2, 31.8]",From Counter,"[120.0, 43.7, 0.0]",0.159284,Open Play,NaN
3,3857256,1,43,41,Switzerland,Switzerland,Breel-Donald Embolo,Center Forward,Shot,Goal,"[114.4, 36.9]",Regular Play,"[120.0, 37.8, 0.6]",0.726976,Open Play,NaN
4,3857256,2,47,38,Switzerland,Switzerland,Remo Freuler,Right Defensive Midfield,Shot,Goal,"[107.2, 41.2]",From Free Kick,"[120.0, 42.7, 0.6]",0.153815,Open Play,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
164,3857261,2,50,53,England,England,Philip Foden,Left Wing,Shot,Goal,"[114.5, 34.1]",Regular Play,"[120.0, 38.7, 0.2]",0.440301,Open Play,NaN
165,3857261,2,67,40,England,England,Marcus Rashford,Right Wing,Shot,Goal,"[109.9, 50.3]",From Throw In,"[120.0, 41.4, 0.2]",0.096603,Open Play,NaN
166,3857255,1,11,0,Spain,Spain,Álvaro Borja Morata Martín,Center Forward,Shot,Goal,"[115.3, 38.3]",Regular Play,"[120.0, 40.3, 0.5]",0.331067,Open Play,NaN
167,3857255,2,47,48,Japan,Japan,Ritsu Doan,Right Wing,Shot,Goal,"[100.5, 55.8]",Regular Play,"[120.0, 42.2, 1.9]",0.027778,Open Play,NaN


In [ ]:
# goal: we get the goal paths fully for every goal into p5 and can then do custom filtering, vis, etc.